In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [ ]:
%%writefile activation_functions.h
#ifndef ACTIVATION_FUNCTIONS_H
#define ACTIVATION_FUNCTIONS_H

float sigmoid_cpu(float x);
__device__ float sigmoid(float x);
__global__ void sigmoid_kernel(float *input, float *output);

void sigmoid(float *input, float *output);
#endif  // ACTIVATION_FUNCTIONS_H


Overwriting activation_functions.h


In [ ]:
%%writefile feedforward.h
#ifndef FEEDFORWARD_H
#define FEEDFORWARD_H


void feedforward_cpu(float *input, float *output, float *weights, float *biases, int input_size, int output_size);
__global__ void feedforward(float *input, float *output, float *weights, float *biases, int input_size, int output_size);

#endif  // FEEDFORWARD_H


Overwriting feedforward.h


In [ ]:
%%writefile initialization.h
#ifndef INITIALIZATION_H
#define INITIALIZATION_H

__global__ void initialize_floats(float *data, int size, float scale);
__global__ void initialize_biases(float *biases, int size);

#endif  // INITIALIZATION_H


Overwriting initialization.h


In [ ]:
%%writefile utilities.h
#ifndef UTILITIES_H
#define UTILITIES_H

#include <cuda_runtime_api.h>

void checkCudaError(cudaError_t error, const char* task);

#endif  // UTILITIES_H


Overwriting utilities.h


In [ ]:
%%writefile activation_functions.cu

#include "activation_functions.h"
#include <math.h>

__device__ float sigmoid(float x) {
    return 1.0 / (1.0 + exp(-x));
}

float sigmoid_cpu(float x) {
    return 1.0 / (1.0 + exp(-x));
}

// Define a CUDA kernel
__global__ void sigmoid_kernel(float *input, float *output) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < 1000) {
        output[idx] = sigmoid(input[idx]);
    }
}

// Define a function to launch the CUDA kernel
void sigmoid(float *input, float *output) {
    int numThreads = 256;
    int numBlocks = (1000 + numThreads - 1) / numThreads;
    sigmoid_kernel<<<numBlocks, numThreads>>>(input, output);
    cudaDeviceSynchronize();
}



Overwriting activation_functions.cu


In [ ]:
%%writefile feedforward.cu
#include "feedforward.h"
#include "activation_functions.h"



void feedforward_cpu(float *input, float *output, float *weights, float *biases, int input_size, int output_size) {
    // Implementation here
    for (int j = 0; j < output_size; j++) {
        float sum = 0;
        for (int i = 0; i < input_size; i++) {
            sum += input[i] * weights[j * input_size + i];
        }
        output[j] = sigmoid_cpu(sum + biases[j]);
    }
}

__global__ void feedforward(float *input, float *output, float *weights, float *biases, int input_size, int output_size) {
    // Implementation here
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < output_size) {
        float sum = 0;
        for (int i = 0; i < input_size; i++) {
            sum += input[i] * weights[idx * input_size + i];
        }
        output[idx] = sigmoid(sum + biases[idx]);
    }
}


Overwriting feedforward.cu


In [ ]:
%%writefile initialization.cu
#include "initialization.h"
#include <curand.h>
#include <curand_kernel.h>

// GPU initialization kernel for weights and input
__global__ void initialize_floats(float *data, int size, float scale) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        curandState state;
        curand_init(5678, idx, 0, &state);
        data[idx] = curand_uniform(&state) * scale;
    }
}

// GPU initialization kernel for biases
__global__ void initialize_biases(float *biases, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        biases[idx] = 0.0;
    }
}


Overwriting initialization.cu


In [ ]:
%%writefile utilities.cu
#include "utilities.h"
#include <stdio.h>

void checkCudaError(cudaError_t error, const char* task) {
    if (error != cudaSuccess) {
        printf("CUDA error after %s: %s\n", task, cudaGetErrorString(error));
        exit(1);
    }
}


Overwriting utilities.cu


In [ ]:
%%writefile main.cu
#include <stdio.h>
#include <chrono>
#include "activation_functions.h"
#include "feedforward.h"
#include "initialization.h"
#include "utilities.h"

int main() {
    int input_neurons = 784;
    int hidden_neurons = 2048;
    int output_neurons = 10;

    // Memory allocation for input, output, weights, and biases
    float *d_input, *d_hidden_output, *d_output;
    float *d_weights_input_to_hidden, *d_weights_hidden_to_output;
    float *d_biases_hidden, *d_biases_output;
    float *h_input = new float[input_neurons];
    float *h_output = new float[output_neurons];
    float *h_hidden_output = new float[hidden_neurons];
    float *h_weights_input_to_hidden = new float[input_neurons * hidden_neurons];
    float *h_weights_hidden_to_output = new float[hidden_neurons * output_neurons];
    float *h_biases_hidden = new float[hidden_neurons];
    float *h_biases_output = new float[output_neurons];

    checkCudaError(cudaMalloc(&d_input, input_neurons * sizeof(float)), "allocating d_input");
    checkCudaError(cudaMalloc(&d_hidden_output, hidden_neurons * sizeof(float)), "allocating d_hidden_output");
    checkCudaError(cudaMalloc(&d_output, output_neurons * sizeof(float)), "allocating d_output");
    checkCudaError(cudaMalloc(&d_weights_input_to_hidden, input_neurons * hidden_neurons * sizeof(float)), "allocating d_weights_input_to_hidden");
    checkCudaError(cudaMalloc(&d_weights_hidden_to_output, hidden_neurons * output_neurons * sizeof(float)), "allocating d_weights_hidden_to_output");
    checkCudaError(cudaMalloc(&d_biases_hidden, hidden_neurons * sizeof(float)), "allocating d_biases_hidden");
    checkCudaError(cudaMalloc(&d_biases_output, output_neurons * sizeof(float)), "allocating d_biases_output");

    // Initialize input, weights, and biases on GPU
    int threadsPerBlock = 256;
    int blocksPerGrid = (input_neurons + threadsPerBlock - 1) / threadsPerBlock;
    initialize_floats<<<blocksPerGrid, threadsPerBlock>>>(d_input, input_neurons, 1.0);
    blocksPerGrid = (input_neurons * hidden_neurons + threadsPerBlock - 1) / threadsPerBlock;
    initialize_floats<<<blocksPerGrid, threadsPerBlock>>>(d_weights_input_to_hidden, input_neurons * hidden_neurons, 0.1);
    blocksPerGrid = (hidden_neurons * output_neurons + threadsPerBlock - 1) / threadsPerBlock;
    initialize_floats<<<blocksPerGrid, threadsPerBlock>>>(d_weights_hidden_to_output, hidden_neurons * output_neurons, 0.1);
    blocksPerGrid = (hidden_neurons + threadsPerBlock - 1) / threadsPerBlock;
    initialize_biases<<<blocksPerGrid, threadsPerBlock>>>(d_biases_hidden, hidden_neurons);
    blocksPerGrid = (output_neurons + threadsPerBlock - 1) / threadsPerBlock;
    initialize_biases<<<blocksPerGrid, threadsPerBlock>>>(d_biases_output, output_neurons);

    // Setup CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Copy weights and biases back to CPU for CPU computation
    cudaMemcpy(h_input, d_input, input_neurons * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_weights_input_to_hidden, d_weights_input_to_hidden, input_neurons * hidden_neurons * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_weights_hidden_to_output, d_weights_hidden_to_output, hidden_neurons * output_neurons * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_biases_hidden, d_biases_hidden, hidden_neurons * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_biases_output, d_biases_output, output_neurons * sizeof(float), cudaMemcpyDeviceToHost);


    // CPU timing
    auto cpu_start = std::chrono::high_resolution_clock::now();
    feedforward_cpu(h_input, h_hidden_output, h_weights_input_to_hidden, h_biases_hidden, input_neurons, hidden_neurons);
    feedforward_cpu(h_hidden_output, h_output, h_weights_hidden_to_output, h_biases_output, hidden_neurons, output_neurons);
    auto cpu_stop = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double, std::milli> cpu_duration = cpu_stop - cpu_start;

    printf("CPU processing time: %f milliseconds\n", cpu_duration.count());

    // Launch kernel for the hidden layer
    cudaEventRecord(start);
    feedforward<<<blocksPerGrid, threadsPerBlock>>>(d_input, d_hidden_output, d_weights_input_to_hidden, d_biases_hidden, input_neurons, hidden_neurons);

    // Launch kernel for the output layer
    feedforward<<<blocksPerGrid, threadsPerBlock>>>(d_hidden_output, d_output, d_weights_hidden_to_output, d_biases_output, hidden_neurons, output_neurons);
    cudaEventRecord(stop);

    // Copy result back to host and print GPU output
    cudaMemcpy(h_output, d_output, output_neurons * sizeof(float), cudaMemcpyDeviceToHost);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    printf("\n");

    for (int i = 0; i < output_neurons; i++) {
        printf("Output[%d]: %f\n", i, h_output[i]);
    }

    printf("GPU processing time: %f milliseconds\n", milliseconds);

    // Cleanup
    cudaFree(d_input);
    cudaFree(d_hidden_output);
    cudaFree(d_output);
    cudaFree(d_weights_input_to_hidden);
    cudaFree(d_weights_hidden_to_output);
    cudaFree(d_biases_hidden);
    cudaFree(d_biases_output);
    delete[] h_input;
    delete[] h_output;
    delete[] h_hidden_output;
    delete[] h_weights_input_to_hidden;
    delete[] h_weights_hidden_to_output;
    delete[] h_biases_hidden;
    delete[] h_biases_output;

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Overwriting main.cu


In [ ]:
!nvcc main.cu activation_functions.cu feedforward.cu initialization.cu utilities.cu --relocatable-device-code true -lcudadevrt -o main_cuda
!./main_cuda


CPU processing time: 7.598326 milliseconds

Output[0]: 0.999995
Output[1]: 0.999997
Output[2]: 0.999996
Output[3]: 0.999998
Output[4]: 0.999997
Output[5]: 0.999996
Output[6]: 0.999996
Output[7]: 0.999998
Output[8]: 0.999996
Output[9]: 0.999997
GPU processing time: 0.533728 milliseconds


## With backpropagation

In [1]:
import numpy as np
import cupy as cp
from numba import cuda

class CUDANeuralNetwork:
    def __init__(self, layers, activation='relu'):
        self.layers = layers
        self.weights = []
        self.biases = []
        self.activation = activation

        for i in range(1, len(layers)):
            self.weights.append(cp.random.randn(layers[i], layers[i-1]).astype(cp.float32) / cp.sqrt(layers[i-1]))
            self.biases.append(cp.zeros((layers[i], 1), dtype=cp.float32))

    def forward(self, X):
        self.activations = [X]
        self.zs = []
        for i in range(len(self.layers) - 1):
            z = cp.dot(self.weights[i], self.activations[-1]) + self.biases[i]
            self.zs.append(z)
            a = self.apply_activation(z)
            self.activations.append(a)
        return self.activations[-1]

    def apply_activation(self, x):
        if self.activation == 'relu':
            return cp.maximum(0, x)
        elif self.activation == 'sigmoid':
            return 1 / (1 + cp.exp(-x))
        elif self.activation == 'tanh':
            return cp.tanh(x)

    def activation_derivative(self, x):
        if self.activation == 'relu':
            return cp.where(x > 0, 1, 0)
        elif self.activation == 'sigmoid':
            s = self.apply_activation(x)
            return s * (1 - s)
        elif self.activation == 'tanh':
            return 1 - cp.tanh(x)**2

    @staticmethod
    @cuda.jit
    def parallel_forward(weights, biases, inputs, outputs, activation):
        i, j = cuda.grid(2)
        if i < outputs.shape[0] and j < outputs.shape[1]:
            tmp = 0
            for k in range(inputs.shape[0]):
                tmp += weights[i, k] * inputs[k, j]
            tmp += biases[i, 0]

            # Apply activation function
            if activation == 0:  # ReLU
                outputs[i, j] = max(0, tmp)
            elif activation == 1:  # Sigmoid
                outputs[i, j] = 1 / (1 + cuda.libdevice.exp(-tmp))
            elif activation == 2:  # Tanh
                outputs[i, j] = cuda.libdevice.tanh(tmp)

    def cuda_forward(self, X):
        X_gpu = cuda.to_device(X)
        self.activations_gpu = [X_gpu]

        for i in range(len(self.layers) - 1):
            weights_gpu = cuda.to_device(self.weights[i].get())
            biases_gpu = cuda.to_device(self.biases[i].get())
            output_shape = (self.layers[i+1], X.shape[1])
            output_gpu = cuda.device_array(output_shape, dtype=np.float32)

            threads_per_block = (16, 16)
            blocks_per_grid = (
                (output_shape[0] + threads_per_block[0] - 1) // threads_per_block[0],
                (output_shape[1] + threads_per_block[1] - 1) // threads_per_block[1]
            )

            activation_code = 0 if self.activation == 'relu' else 1 if self.activation == 'sigmoid' else 2

            self.parallel_forward[blocks_per_grid, threads_per_block](
                weights_gpu, biases_gpu, self.activations_gpu[-1], output_gpu, activation_code
            )

            self.activations_gpu.append(output_gpu)

        return self.activations_gpu[-1].copy_to_host()

    def backpropagation(self, X, y, learning_rate=0.01):
        m = X.shape[1]

        # Forward pass
        self.forward(X)

        # Backward pass
        delta = self.activations[-1] - y
        nabla_b = [cp.zeros(b.shape) for b in self.biases]
        nabla_w = [cp.zeros(w.shape) for w in self.weights]

        nabla_b[-1] = cp.sum(delta, axis=1, keepdims=True) / m
        nabla_w[-1] = cp.dot(delta, self.activations[-2].T) / m

        for l in range(2, len(self.layers)):
            delta = cp.dot(self.weights[-l+1].T, delta) * self.activation_derivative(self.zs[-l])
            nabla_b[-l] = cp.sum(delta, axis=1, keepdims=True) / m
            nabla_w[-l] = cp.dot(delta, self.activations[-l-1].T) / m

        # Update weights and biases
        self.weights = [w - learning_rate * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - learning_rate * nb for b, nb in zip(self.biases, nabla_b)]

    def train(self, X, y, epochs=100, learning_rate=0.01, batch_size=32):
        for epoch in range(epochs):
            for i in range(0, X.shape[1], batch_size):
                X_batch = X[:, i:i+batch_size]
                y_batch = y[:, i:i+batch_size]
                self.backpropagation(X_batch, y_batch, learning_rate)

            if epoch % 10 == 0:
                loss = self.compute_loss(X, y)
                print(f"Epoch {epoch}, Loss: {loss}")

    def compute_loss(self, X, y):
        output = self.forward(X)
        return cp.mean(cp.sum((output - y)**2, axis=0))

# Example usage
if __name__ == "__main__":
    # Define the neural network architecture
    layers = [784, 128, 64, 10]  # Example for MNIST dataset

    # Create the neural network
    nn = CUDANeuralNetwork(layers, activation='sigmoid')

    # Generate random input data and labels
    input_data = np.random.randn(784, 1000).astype(np.float32)  # 1000 samples
    labels = np.random.randint(0, 10, size=(10, 1000)).astype(np.float32)
    labels = (labels == np.arange(10)[:, np.newaxis]).astype(np.float32)

    # Train the network
    nn.train(cp.asarray(input_data), cp.asarray(labels), epochs=100, learning_rate=0.1)

    # Perform forward pass
    output = nn.cuda_forward(input_data)
    print("Output shape:", output.shape)

Epoch 0, Loss: 0.8870215449612883
Epoch 10, Loss: 0.8657606718556794
Epoch 20, Loss: 0.7732219297377405
Epoch 30, Loss: 0.47857357297836933
Epoch 40, Loss: 0.19293552188522464
Epoch 50, Loss: 0.06031313058716237
Epoch 60, Loss: 0.0215947776557352
Epoch 70, Loss: 0.009863558819462692
Epoch 80, Loss: 0.005371389576487265
Epoch 90, Loss: 0.003290592687824068
Output shape: (10, 1000)


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 63 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
